# https://www.kaggle.com/code/awqatak/silver-bullet-single-model-165-features#Polars-FE-&-Helper-Functions

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import re
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings("ignore")

In [2]:
is_kaggle = False

if is_kaggle:
    base_dir = '/kaggle/input'
    data_dir = f'{base_dir}/linking-writing-processes-to-writing-quality'
    output_dir = '/kaggle/working'
else:
    base_dir = '../'
    data_dir = f'{base_dir}/data'
    models_dir = f'{base_dir}/models'

## Polars FE & Helper Functions

In [3]:
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter((pl.col('time_diff') > 3) & (pl.col('time_diff') < 5)).count(),
                                   pauses_5_sec = pl.col('time_diff').filter(pl.col('time_diff') > 5).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    

    print("< Cursor moving animation >")

    # Adding 'pos' and 'line' columns
    df = df.with_columns([
        (pl.col('cursor_position') % 30).alias('pos'),
        (pl.col('cursor_position') / 30).cast(pl.Int32).alias('line')
    ])

    # Adding 'dist_moved' column
    df = df.with_columns(
        [pl.col('cursor_position').diff().over('id').fill_null(0).abs().alias('dist_moved')]
    )

    # Calculating average distance moved per event
    avg_dist_per_event = df.groupby('id').agg(
        pl.mean('dist_moved').alias('avg_dist_per_event')
    )

    # Joining with the feature set
    feats = feats.join(avg_dist_per_event, on='id', how='left')

    # Adding 'line_change' column
    df = df.with_columns(
        [pl.col('line').diff().over('id').fill_null(0).ne(0).alias('line_change')]
    )

    # Calculating revisits per line and first line revisits
    revisits_per_line = df.filter(pl.col('line_change') == False).group_by(['id', 'line']).agg(
        pl.count().alias('revisit_count')
    )
    first_line_revisits = revisits_per_line.filter(pl.col('line') == 0).group_by('id').agg(
        pl.sum('revisit_count').alias('first_line_revisits')
    )

    # Joining with the feature set
    feats = feats.join(first_line_revisits, on='id', how='left')

    # Calculating line changes
    line_changes = df.groupby('id').agg(
        pl.sum('line_change').alias('line_changes')
    )

    # Joining with the feature set
    feats = feats.join(line_changes, on='id', how='left')

    return feats

## Pandas FE & Helper Functions

In [4]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum'] #, 'std', 'var', 'sem']

def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df


def word_feats(df):
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
    df = df[df.sent_len!=0].reset_index(drop=True)

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                             df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df


def calculate_relative_paragraph_sizes(input_df):

    df = input_df.copy()
    df['total_paragraphs'] = df['paragraph'].apply(len)

    df['relative_intro_size'] = 1 / df['total_paragraphs']  # First paragraph is the introduction
    df['relative_body_size'] = (df['total_paragraphs'] - 2) / df['total_paragraphs']  # Middle paragraphs are the body
    df['relative_conclusion_size'] = 1 / df['total_paragraphs']  # Last paragraph is the conclusion

    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: [len(paragraph.split()) for paragraph in x])

    df['word_count_intro'] = df['paragraph_word_count'].apply(lambda x: x[0] if len(x) > 0 else 0)
    df['word_count_body'] = df['paragraph_word_count'].apply(lambda x: sum(x[1:-1]) if len(x) > 2 else 0)
    df['word_count_conclusion'] = df['paragraph_word_count'].apply(lambda x: x[-1] if len(x) > 1 else 0)

    df['total_word_count'] = df['paragraph_word_count'].apply(sum)

    df['intro_ratio'] = df['word_count_intro'] / df['total_word_count']
    df['body_ratio'] = df['word_count_body'] / df['total_word_count']
    df['conclusion_ratio'] = df['word_count_conclusion'] / df['total_word_count']

    df['intro_body_ratio'] = df['word_count_intro'] / df['word_count_body']
    df['intro_conclusion_ratio'] = df['word_count_intro'] / df['word_count_conclusion']
    df['body_conclusion_ratio'] = df['word_count_body'] / df['word_count_conclusion']

    df.drop(columns=['paragraph', 'total_paragraphs', 'essay', 'paragraph_word_count', 'word', 'sent'], inplace=True)

    return df


def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]

def count_pauses_2s(group):
    """Counts pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    return (gap > 2000).sum()

def count_pauses_5s(group):
    """Counts pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    return (gap > 5000).sum()

def pause_proportion_2s(group):
    """Calculates the proportion of pause time to total essay time."""
    gap = group['down_time'] - group['up_time'].shift(1)
    total_pause_time = gap[gap > 2000].sum()
    total_essay_time = group['up_time'].max() - group['down_time'].min()
    return total_pause_time / total_essay_time if total_essay_time else 0

def pause_proportion_5s(group):
    """Calculates the proportion of pause time to total essay time."""
    gap = group['down_time'] - group['up_time'].shift(1)
    total_pause_time = gap[gap > 2000].sum()
    total_essay_time = group['up_time'].max() - group['down_time'].min()
    return total_pause_time / total_essay_time if total_essay_time else 0

def mean_pause_length(group):
    """Calculates the mean length of pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    pauses = gap[gap > 2000]
    return pauses.mean() / 1000 if not pauses.empty else 0

def median_pause_length(group):
    """Calculates the mean length of pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    pauses = gap[gap > 2000]
    return pauses.median() / 1000 if not pauses.empty else 0

# Method to aggregate all pause-related features
def aggregate_pause_features(df):
    """Aggregates all pause-related features for each essay."""
    grouped = df.groupby('id')
    pause_features = pd.DataFrame()
    pause_features['n_pauses_2s'] = grouped.apply(count_pauses_2s)
    pause_features['n_pauses_5s'] = grouped.apply(count_pauses_5s)
    pause_features['pause_proportion_2s'] = grouped.apply(pause_proportion_2s)
    pause_features['pause_proportion_5s'] = grouped.apply(pause_proportion_5s)
    pause_features['mean_pause_length'] = grouped.apply(mean_pause_length)
    pause_features['median_pause_length'] = grouped.apply(median_pause_length)
    return pause_features

def process_variance(group):
    """Calculates the variance in the writing process over time for each essay."""
    if len(group) < 2:  # Handling for groups with a single row
        return 0

    bins = np.linspace(group['down_time'].min(), group['up_time'].max(), 11)
    divisions = pd.cut(group['down_time'], bins=bins, include_lowest=True, labels=range(1, 11))
    production_deciles = group.groupby(divisions).agg(n_events=('event_id', 'count'))
    return np.std(production_deciles['n_events'], ddof=1)

def aggregate_process_variance(df):
    """Aggregates the process variance feature for each essay."""
    return df.groupby('id').apply(process_variance).rename('process_variance').to_frame()

def calculate_segment_visits(train_logs, train_essays):
    # Calculate the end position of the intro and body for each essay
    train_essays['intro_end'] = train_essays['essay'].apply(lambda x: len(x.split('\n')[0]))
    train_essays['body_end'] = train_essays['essay'].apply(lambda x: len('\n'.join(x.split('\n')[:-1])))

    # Create a dictionary for quick lookup
    ends_dict = train_essays.set_index('id')[['intro_end', 'body_end']].to_dict('index')

    # Function to categorize position
    def categorize_position(row):
        intro_end = ends_dict[row['id']]['intro_end']
        body_end = ends_dict[row['id']]['body_end']
        if row['cursor_position'] <= intro_end:
            return 'intro'
        elif row['cursor_position'] <= body_end:
            return 'body'
        else:
            return 'conclusion'

    # Categorize cursor positions
    train_logs['segment'] = train_logs.apply(categorize_position, axis=1)

    # Count visits to each segment
    segment_visits = train_logs.groupby(['id', 'segment']).size().unstack(fill_value=0)

    # Rename columns for clarity
    segment_visits.columns = [f'{col}_visits' for col in segment_visits.columns]

    return segment_visits

## Solution

In [5]:
# train_logs    = pl.scan_csv(data_dir + '/train_logs.csv')
train_logs    = pl.scan_csv('./train_logs_corrected.csv')
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()

print('< Essay Reconstruction >')
train_logs             = train_logs.collect().to_pandas()
train_essays           = get_essay_df(train_logs)
train_feats            = train_feats.merge(word_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(sent_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(parag_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(calculate_relative_paragraph_sizes(train_essays), on='id', how='left')
train_feats            = train_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
train_feats            = train_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')
train_feats            = train_feats.merge(aggregate_pause_features(train_logs).reset_index(), on='id', how='left')
train_feats            = train_feats.merge(aggregate_process_variance(train_logs).reset_index(), on='id', how='left')
train_feats            = train_feats.merge(calculate_segment_visits(train_logs, train_essays).reset_index(), on='id', how='left')

print('< Mapping >')
train_scores   = pd.read_csv(data_dir + '/train_scores.csv')
data           = train_feats.merge(train_scores, on='id', how='left')
x              = data.drop(['id', 'score'], axis=1)
y              = data['score'].values
print(f'Number of features: {len(x.columns)}')

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Cursor moving animation >
< Essay Reconstruction >
< Mapping >
Number of features: 192


In [6]:
import pandas as pd
import numpy as np

def filter_columns(df):
    # Initialize lists to store column names for each condition
    all_nan_columns = []
    all_inf_columns = []
    single_value_columns = []

    # Iterate through columns
    for col in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[col]):
            # Check for all NaN values
            if df[col].isna().all():
                all_nan_columns.append(col)
            else:
                # Check for all infinite values (positive or negative infinity)
                if np.isinf(df[col]).all():
                    all_inf_columns.append(col)
                else:
                    # Check for more than 90% of columns with only one unique value
                    unique_value_count = df[col].nunique()
                    total_count = df.shape[0]
                    if unique_value_count == 1 and (total_count - df[col].isna().sum()) / total_count > 0.8:
                        single_value_columns.append(col)

    return {
        "AllNaNColumns": all_nan_columns,
        "AllInfColumns": all_inf_columns,
        "SingleValueColumns": single_value_columns
    }

# Example usage:
# Assuming 'df' is your DataFrame
filtered_columns = filter_columns(data)
print("Columns with all NaNs:", filtered_columns["AllNaNColumns"])
print("Columns with all infinite values:", filtered_columns["AllInfColumns"])
print("Columns with more than 90% of the same value:", filtered_columns["SingleValueColumns"])

Columns with all NaNs: []
Columns with all infinite values: []
Columns with more than 90% of the same value: ['cursor_position_min']


In [7]:
data.to_csv('silver_bullet_192feat_corrected.csv', index=False)